In [ ]:
#This code trains a random forest classifier and reflects the experiment carried out in Part II of the paper (appendix B.1). 
#This code uses no statistical properties and thus fails to achieve high accuracy. Accuracy here: 0.619

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers  import Dense, BatchNormalization, Activation
from sklearn.metrics  import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.callbacks  import EarlyStopping
from tensorflow.keras.models   import Sequential

In [ ]:
train_file = '/content/drive/MyDrive/Dirichlet Mod Conh/zeros_train.csv' #change this to use
test_file  = '/content/drive/MyDrive/Dirichlet Mod Conh/zeros_test.csv' #change this to use
nb_of_zeros=25
cols = ['y'] + [f'zero_{i+1}' for i in range(nb_of_zeros)] # 25 is the number of zeros per line

In [ ]:
def load_data(t_path):
    t_data = []
    with open(t_path) as f:
        for line in f:
            label, zeros = line.split(':')
            y = int(label)
            zeros = [float(zero) for zero in zeros.split(',') if zero.strip()]
            t_data.append([y] + zeros)
    return pd.DataFrame(t_data, columns=cols)

train  = load_data(train_file)
test   = load_data(test_file)

In [ ]:
X_train = train.iloc[:,1:].values
y_train = train['y'].values
X_test  = test .iloc[:,1:].values
y_test  = test ['y'].values

In [ ]:
#to ensure result is independent of how the labels are ordered in datasets we shuffle them

rand= np.random.RandomState(42)
permutation1 = rand.permutation(len(y_train))
X_train, y_train = X_train[permutation1], y_train[permutation1]
permutation2 = rand.permutation(len(y_test))
X_test,  y_test  = X_test[permutation2],  y_test[permutation2]

In [ ]:
#standardize and ecoding
scaler   = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler  = scaler.transform(X_test)

label_ecode = LabelEncoder()
y_train_encode = label_ecode.fit_transform(y_train)
y_test_encode  = label_ecode.transform(y_test)

In [ ]:
random_forest_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
random_forest_model.fit(X_train_scaler, y_train_encode)


In [ ]:
random_forest_test_pred  = random_forest_model.predict_proba(X_test_scaler)
random_forest_test_pred_labels = label_ecode.inverse_transform(random_forest_test_pred)
accuracy = accuracy_score(y_test, random_forest_test_pred_labels)

In [ ]:
print(f"Test accuracy: {accuracy:.4f}")
print("y_test")
print(y_test)
print("y_pred")
print(random_forest_test_pred_labels)